In [1]:
!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, db
from datetime import datetime  # Untuk konversi timestamp ke human-readable
import pandas as pd


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Konfigurasi database sumber
source_cred = credentials.Certificate("/work/database-sensor-iklim-litbang-firebase-admin.json")
firebase_admin.initialize_app(source_cred, {
    'databaseURL': 'https://database-sensor-iklim-litbang-default-rtdb.asia-southeast1.firebasedatabase.app/'
})

In [3]:
# Referensi ke data sumber
source_ref = db.reference('/auto_weather_stat/id-04/data')

# Mengambil data dari database sumber
source_data = source_ref.get()

# Mengubah data menjadi DataFrame pandas
if source_data:
    # Mengubah data menjadi DataFrame
    df = pd.DataFrame.from_dict(source_data, orient='index')
    
    # Jika 'timestamp' sudah ada, jangan pindahkan indeks ke kolom
    if 'timestamp' not in df.columns:
        df.index.name = 'timestamp'  # Mengatur nama indeks
        df.reset_index(inplace=True)  # Memindahkan indeks menjadi kolom biasa
    
    print(df.head())  # Menampilkan data untuk verifikasi
else:
    print("Tidak ada data di jalur /auto_weather_stat/id-03/data.")

                 dew  humidity  pressure  temperature   timestamp     volt
0           29.21617  84.65000  1002.930     32.14000           0  3.91250
1732280466  29.33488  98.57198  1007.966     29.58495  1732280466  4.02250
1732280481  29.32663  98.63111  1007.955     29.56626  1732280481  4.02250
1732280496  29.35434  98.66735  1007.957     29.58762  1732280496  4.02375
1732280511  29.35272  98.70359  1007.949     29.57961  1732280511  4.02500


In [4]:
df.head(10)

,dew,humidity,pressure,temperature,timestamp,volt
0,29.21617,84.65000,1002.930,32.14000,0,3.91250
1732280466,29.33488,98.57198,1007.966,29.58495,1732280466,4.02250
1732280481,29.32663,98.63111,1007.955,29.56626,1732280481,4.02250
1732280496,29.35434,98.66735,1007.957,29.58762,1732280496,4.02375
1732280511,29.35272,98.70359,1007.949,29.57961,1732280511,4.02500
1732280526,29.36880,98.76463,1007.956,29.58495,1732280526,4.02375
1732280541,29.37081,98.77608,1007.962,29.58495,1732280541,4.02625
1732280556,29.36786,98.81995,1007.955,29.57427,1732280556,4.02375
1732280586,29.37687,98.82567,1007.945,29.58228,1732280586,4.02125
1732280625,29.36459,98.87717,1007.956,29.56092,1732280625,4.02125


In [5]:
# Input readable date
readable_date = "01-04-2025 11:15:00"  # Format: DD-MM-YYYY HH:MM:SS

# Convert readable date to Unix timestamp
start_timestamp = int(datetime.strptime(readable_date, "%d-%m-%Y %H:%M:%S").timestamp())
print(start_timestamp)
end_timestamp = int(datetime.now().timestamp())  # Current time in Unix timestamp

# Filter data berdasarkan rentang waktu dengan validasi kunci
filtered_data = {
    key: value
    for key, value in source_data.items()
    if key.isdigit() and start_timestamp <= int(key) <= end_timestamp
}

1743506100


In [6]:
# Konfigurasi database tujuan
dest_cred = credentials.Certificate("/work/staklimjerukagung-firebase-admin.json")
dest_app = firebase_admin.initialize_app(dest_cred, {
    'databaseURL': 'https://staklimjerukagung-default-rtdb.asia-southeast1.firebasedatabase.app/'
}, name='destination')

In [7]:
# Referensi ke data tujuan
dest_ref = db.reference('/auto_weather_stat/id-04/data', app=dest_app)

In [8]:
# Memindahkan data yang telah difilter ke database tujuan
if filtered_data:
    dest_ref.update(filtered_data)
    print("Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.")
else:
    print("Tidak ada data dalam rentang waktu yang ditentukan.")

Data berhasil dipindahkan ke /auto_weather_stat/id-03 di database tujuan.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=200afad5-8991-4e20-8a55-8751c7aff3b5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>